# Build a motion model from feature matches

## First, get keypoints

Features and neurons

In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *

from tqdm import tqdm

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

# Actually import
import_opt = {'num_slices':33, 'alpha':0.15}

dat0_vid = get_single_volume(btf_fname_red, 15, **import_opt)
dat1_vid = get_single_volume(btf_fname_red, 30, **import_opt)

## Segment 2 volumes

In [3]:
opt = {'num_slices':33, 'alpha':1.0}
# Build point clouds for each plane
all_keypoints_pcs0 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp0 = build_correspondence_icp(all_keypoints_pcs0)

all_keypoints_pcs1 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp1 = build_correspondence_icp(all_keypoints_pcs1)

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


In [4]:
all_neurons = [k.points for k in all_keypoints_pcs0]
all_matches = [m.correspondence_set for m in all_icp0]
clust_df0 = build_tracklets_from_matches(all_neurons, all_matches)

all_neurons = [k.points for k in all_keypoints_pcs1]
all_matches = [m.correspondence_set for m in all_icp1]
clust_df1 = build_tracklets_from_matches(all_neurons, all_matches)

In [5]:
opt = {'num_slices':33, 'alpha':1.0, 'verbose':1}
neurons0, df0, icp0, pcs0 = detect_neurons_using_ICP(dat0_vid, **opt)
neurons1, df1, icp1, pcs1 = detect_neurons_using_ICP(dat1_vid, **opt)

all_features0, all_features1, kp0, kp1, feature_matches = build_features_and_match_2volumes(dat0_vid,dat1_vid,
                                                                      verbose=1,
                                                                      matches_to_keep=0.8,
                                                                      num_features_per_plane=10000,
                                                                      detect_keypoints=True,
                                                                      kp0=neurons0,
                                                                      kp1=neurons1)

all_matches, f2n, all_conf = match_centroids_using_tree(np.array(neurons0), 
                               np.array(neurons1), 
                               all_features0, 
                               all_features1,
                               radius=8,
                               max_nn=50,
                               min_features_needed=5,
                                   verbose=1,
                                     to_mirror=False)

Building pairwise correspondence...
Building clusters...
Finished ID'ing neurons
Building pairwise correspondence...
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
Building clusters...
Finished ID'ing neurons
Removed the following duplicates: [106, 105, 104, 97, 93, 83, 82, 78, 73, 70, 69, 67, 64, 62, 61, 51, 44, 35, 32, 31, 29, 28, 21, 15, 9]


## Alternate way of getting keypoints: my Frame class

In [89]:
opt = {'start_frame':0,
       'num_frames':5,
      'num_reference_frames':4,
      'start_slice':4}
all_matches, all_other_frames, reference_set = track_via_reference_frames(btf_fname_red, **opt)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.62s/it]


In [27]:
print(reference_set)

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 202 

                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 187 

                RegisteredReferenceFrames:
                Number of frames: 2 



## An example of a full affine model

In [37]:
pts0_unmatched = reference_set.reference_frames[0].keypoint_locs
pts1_unmatched = reference_set.reference_frames[1].keypoint_locs

feature_matches = reference_set.feature_matches[(0,1)]

In [38]:
# Align the keypoints via matches

pts0 = np.zeros((len(feature_matches), 3), dtype=np.float32)
pts1 = np.zeros((len(feature_matches), 3), dtype=np.float32)
for m, match in enumerate(feature_matches):
    pts0[m, :] = pts0_unmatched[match.queryIdx]
    pts1[m, :] = pts1_unmatched[match.trainIdx]

In [39]:
# h = np.transpose(h)
# Note: neurons0 is a dataframe
# n0_2d = np.array([np.array([n[1], n[2]]) for n in neurons0])

val, h, inliers = cv2.estimateAffine3D(pts0,pts1, confidence=0.99)
pts0_trans = cv2.transform(np.array([pts0]), h)[0]

In [40]:
h

array([[ 9.15033187e-01, -2.96604776e-02, -2.51493568e-03,
         1.07420336e+01],
       [-1.99451570e-01,  8.03839099e-01, -6.27004957e-01,
         2.64056234e+02],
       [-1.46517814e-01,  5.14687677e-01,  7.89790389e-01,
        -1.47872252e+02]])

In [57]:
# Make point clouds
pc0 = o3d.geometry.PointCloud()
pc0.points = o3d.utility.Vector3dVector(pts0)
pc0.paint_uniform_color([0,0,0])

pc0_trans = o3d.geometry.PointCloud()
pc0_trans.points = o3d.utility.Vector3dVector(pts0_trans)
pc0_trans.paint_uniform_color([0.5,0.5,0.5])

pc1 = o3d.geometry.PointCloud()
pc1.points = o3d.utility.Vector3dVector(pts1)
pc1.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([pc0,pc0_trans, pc1])

## Next, use the affine model locally

In [94]:
i0, i1 = 0, 1

f0 = reference_set.reference_frames[i0]
f1 = reference_set.reference_frames[i1]
all_feature_matches = reference_set.feature_matches[(i0, i1)]

In [97]:
all_lines = None
all_trans = None

for which_neuron in tqdm(range(len(f0.neuron_locs))):
    success, neuron0_trans = propagate_via_affine_model(which_neuron, f0, f1, all_feature_matches)
    if not success:
        continue
    pc0_neuron, pc1_trans, pc1_neuron, line = create_affine_visualizations(which_neuron, f0, f1, neuron0_trans)

    if all_lines is None:
        all_lines = line
        all_trans = pc1_trans
    else:
        all_lines = all_lines + line
        all_trans = all_trans + pc1_trans

100%|████████████████████████████████████████████████████████████████████████████████| 202/202 [00:04<00:00, 42.01it/s]


In [98]:
o3d.visualization.draw_geometries([pc0_neuron, all_trans, pc1_neuron, all_lines])

# Finally, turn this into a set of matches

There are three obvious forms of mismatch:
1. Missing neuron in either v0 or v1
2. Related, multiple v0 neurons near one v1 neuron
3. Neuron pushed to an ambiguous location, not very close to a single v1 neuron

To some extent this just needs better neuron segmentation. However, the features are what they are; theoretically the v0 neurons should be pushed close to a v1 neuron, even if it is missing

Strategy 1: greedy
1. Loop over v1 neurons, taking the closest one

--- FOR NOW DO THIS:

Strategy 1.5: greedy with best of 2
1. Loop over v1 neurons, taking the closest two
2. If the closest one is significantly closer, only keep that one


Strategy 2: bipartite
1. Loop over v0 or v1 neurons, getting distances to ~2 closest neighbors
2. Bipartite matching on resulting graph

In [99]:
# Build tree to query v1 neurons
tree_n1 = o3d.geometry.KDTreeFlann(pc1_neuron)

In [126]:
# Loop over locations of pushed v0 neurons
all_matches = [] # Without confidence
all_conf = []
nn_opt = { 'radius':20.0, 'max_nn':2}
verbose = 1
dist_ratio = 1.5
conf_func = lambda dist : 1.0 / (dist/10+1.0)
for i, neuron in enumerate(np.array(all_trans.points)):
    [k, two_neighbors, two_dist] = tree_n1.search_hybrid_vector_3d(neuron, **nn_opt)
    
    if k==0:
#         print(f"No close neuron {i}")
        continue
    
    if k==1:
        dist = two_dist[0]
        i_match = two_neighbors[0]
    else:
        if two_dist[0]/two_dist[1] > dist_ratio:
            dist = two_dist[1]
            i_match = two_neighbors[1]
        elif two_dist[1]/two_dist[0] > dist_ratio:
            dist = two_dist[1]
            i_match = two_neighbors[1]
        else:
            if verbose >= 2:
                print(f"Neuron {i} has two close neighbors")
            continue
    
    if verbose >= 2:
        print(f"Found good match for neuron {i}")
    all_matches.append([i, i_match])
    all_conf.append(conf_func(dist))

# Full video

In [ ]:
# Now, full fv
out = track_neurons_full_video(btf_fname_red,
                             start_frame=0,
                             num_frames=500,
                             num_slices=33,
                             alpha=0.15,
                             neuron_feature_radius=5.0,
                             do_mini_max_projections=True,
                             use_affine_matching=True)

 79%|█████████████████████████████████████████████████████████████▌                | 394/499 [1:37:15<23:44, 13.57s/it]

In [ ]:
all_matches, all_conf, all_neurons = out
clust_df = build_tracklets_from_matches(all_neurons, all_matches, all_conf, verbose=1)

In [ ]:
import pickle
fname = 'clust_df_dat_affine.pickle'
with open(fname, 'wb') as f:
    pickle.dump(clust_df,f)